# Arguments for benchmarking

!pip install line_profiler

In [ ]:
%load_ext line_profiler

In [11]:
import numpy as np
from shapely.geometry import Point
from pymeos.db.psycopg import MobilityDB

from pymeos import *
import os
import sys
from datetime import timedelta, datetime
from pymeos import *
import time
from tqdm import tqdm
import itertools
import shutil
import math
import subprocess
from collections import deque
import multiprocessing
import logging
from enum import Enum
import psutil

pymeos_initialize()

SRID = 4326
DATABASE_NAME = "mobilitydb"
TPOINT_TABLE_NAME = "PyMEOS_demo"
TPOINT_ID_COLUMN_NAME = "MMSI"
TPOINT_COLUMN_NAME = "trajectory"
x_min = -180
y_min = -90
x_max = 180
y_max = 90
extent_list = [x_min, y_min, x_max, y_max]

class Time_granularity(Enum):
    # MILLISECOND = {"timedelta" : timedelta(milliseconds=1), "qgs_unit" : QgsUnitTypes.TemporalUnit.Milliseconds, "name" : "MILLISECOND"}
    SECOND = {"timedelta" : timedelta(seconds=1), "name" : "SECOND", "steps" : 1}
    MINUTE = {"timedelta" : timedelta(minutes=1), "name" : "MINUTE", "steps" : 1}
    # HOUR = {"timedelta" : timedelta(hours=1), "qgs_unit" : QgsUnitTypes.TemporalUnit.Hours, "name" : "HOUR"}

    @classmethod
    def set_time_step(cls, steps):
        Time_granularity.SECOND.value["timedelta"] = timedelta(seconds=steps)
        Time_granularity.SECOND.value["steps"] = steps
        Time_granularity.MINUTE.value["timedelta"] = timedelta(minutes=steps)
        Time_granularity.MINUTE.value["steps"] = steps
        return cls
    
GRANULARITY = Time_granularity.set_time_step(1).MINUTE

connection_params = {
        "host": "localhost",
        "port": 5432,
        "dbname": DATABASE_NAME,
        "user": "postgres",
        "password": "postgres",
        
        }

connection_parameters = {
                'host': "localhost",
                'port': 5432,
                'dbname': DATABASE_NAME,
                'user': "postgres",
                'password': "postgres",
                'table_name': TPOINT_TABLE_NAME,
                'id_column_name': TPOINT_ID_COLUMN_NAME,
                'tpoint_column_name': TPOINT_COLUMN_NAME,
            }

                
connection = MobilityDB.connect(**connection_params)
cursor = connection.cursor()

cursor.execute(f"SELECT {TPOINT_ID_COLUMN_NAME} FROM public.{TPOINT_TABLE_NAME};")
Total_ids = cursor.fetchall()

cursor.execute(f"SELECT MIN(startTimestamp({TPOINT_COLUMN_NAME})) AS earliest_timestamp FROM public.{TPOINT_TABLE_NAME};")
start_date = cursor.fetchone()[0]
cursor.execute(f"SELECT MAX(endTimestamp({TPOINT_COLUMN_NAME})) AS latest_timestamp FROM public.{TPOINT_TABLE_NAME};")
end_date = cursor.fetchone()[0]

total_frames = math.ceil( (end_date - start_date) // GRANULARITY.value["timedelta"] ) + 1

timestamps = [start_date + i * GRANULARITY.value["timedelta"] for i in range(total_frames)]
timestamps = [dt.replace(tzinfo=None) for dt in timestamps]
timestamps_strings = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in timestamps]

begin_frame = 0
end_frame = 59
TIME_DELTA_SIZE = end_frame - begin_frame + 1

ids_list = Total_ids[:int(len(Total_ids)*1)]
ids_list_str = [ f"'{id[0]}'"  for id in ids_list]
objects_id_str = ', '.join(map(str, ids_list_str))

objects_id_str
TIME_DELTA_SIZE

60

In [12]:
import multiprocessing

result_queue = multiprocessing.Queue()

arguments = [None, begin_frame, end_frame, TIME_DELTA_SIZE, extent_list, timestamps, connection_parameters, GRANULARITY, SRID, objects_id_str]

# Matrix generation memory profiling

In [13]:

def create_matrix( result_queue, begin_frame, end_frame, time_delta_size, extent, timestamps, connection_parameters, granularity_enum, srid, ids_str):
    try:
        logs=""
        pid = os.getpid()
        logs += (f"pid : {pid} create_matrix |  CPU affinity : {psutil.Process(pid).cpu_affinity()} \n") 
        p_start = timestamps[begin_frame]
        p_end = timestamps[end_frame]
        start_date = timestamps[0]
        x_min,y_min, x_max, y_max = extent
        logs = ""
        
        # Part 1 : Fetch Tpoints from MobilityDB database
        connection_params = {
            "host": connection_parameters["host"],
            "port": connection_parameters["port"],
            "dbname": connection_parameters["dbname"],
            "user": connection_parameters["user"],
            "password": connection_parameters["password"]
        }
        table_name = connection_parameters["table_name"]
        id_column_name = connection_parameters["id_column_name"]
        tpoint_column_name = connection_parameters["tpoint_column_name"]

        connection = MobilityDB.connect(**connection_params)    
        cursor = connection.cursor()
    
        if granularity_enum.value["name"] == "SECOND": 
            time_value = 1 * granularity_enum.value["steps"]
        elif granularity_enum.value["name"] == "MINUTE":
            time_value = 60 * granularity_enum.value["steps"]

        query = f"""WITH trajectories as (
                SELECT 
                    atStbox(
                        a.{tpoint_column_name}::tgeompoint,
                        stbox(
                            ST_MakeEnvelope(
                                {x_min}, {y_min}, -- xmin, ymin
                                {x_max}, {y_max}, -- xmax, ymax
                                {srid} -- SRID
                            ),
                            tstzspan('[{p_start}, {p_end}]')
                        )
                    ) as trajectory
                FROM public.{table_name} as a 
                WHERE a.{id_column_name} in ({ids_str})),

                resampled as (

                SELECT tsample(traj.trajectory, INTERVAL '{granularity_enum.value["steps"]} {granularity_enum.value["name"]}', TIMESTAMP '{start_date}')  AS resampled_trajectory
                    FROM 
                        trajectories as traj)
            
                SELECT
                        EXTRACT(EPOCH FROM (startTimestamp(rs.resampled_trajectory) - '{start_date}'::timestamp))::integer / {time_value} AS start_index ,
                        EXTRACT(EPOCH FROM (endTimestamp(rs.resampled_trajectory) - '{start_date}'::timestamp))::integer / {time_value} AS end_index,
                        rs.resampled_trajectory
                FROM resampled as rs ;"""

        cursor.execute(query)
        # logs += f"query : {query}\n"
        rows = cursor.fetchall()
        cursor.close()
        connection.close()

        logs += f"Number of rows : {len(rows)}\n"
        now_matrix =time.time()
        empty_point_wkt = Point().wkt  # "POINT EMPTY"
        matrix = np.full((len(rows), time_delta_size), empty_point_wkt, dtype=object)
        
        
        rows_to_remove = []
        for i in range(len(rows)):
            if rows[i][2] is not None:
                try:
                    traj_resampled = rows[i][2]

                    start_index = rows[i][0] - begin_frame
                    end_index = rows[i][1] - begin_frame
                    values = np.array([point.wkt for point in traj_resampled.values()])
                    matrix[i, start_index:end_index+1] = values
            
                except Exception as e:
                    logs += f"Error at row {i} : {e}\n"
                    break
            else:
                rows_to_remove.append(i)
        
        matrix = np.delete(matrix, rows_to_remove, axis=0) 

        logs += f"Matrix generation time : {time.time() - now_matrix}\n"
        logs += f"Matrix shape : {matrix.shape}\n"
        logs += f"Number of non empty points : {np.count_nonzero(matrix != 'POINT EMPTY')}\n"

        result_queue.put(0)
        result_queue.put(matrix)
        result_queue.put(logs)

    except Exception as e:
        result_queue.put(1)
        result_queue.put(e)
        result_queue.put(logs)
        return False
    

In [14]:
%lprun -f create_matrix create_matrix(result_queue, begin_frame, end_frame, TIME_DELTA_SIZE, extent_list, timestamps, connection_parameters, GRANULARITY, SRID, objects_id_str)

Timer unit: 1e-09 s

Total time: 4.56213 s
File: /tmp/ipykernel_14199/2186246865.py
Function: create_matrix at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def create_matrix( result_queue, begin_frame, end_frame, time_delta_size, extent, timestamps, connection_parameters, granularity_enum, srid, ids_str):
     2         1        370.0    370.0      0.0      try:
     3         1        130.0    130.0      0.0          logs=""
     4         1        870.0    870.0      0.0          pid = os.getpid()
     5         1     509524.0 509524.0      0.0          logs += (f"pid : {pid} create_matrix |  CPU affinity : {psutil.Process(pid).cpu_affinity()} \n") 
     6         1        370.0    370.0      0.0          p_start = timestamps[begin_frame]
     7         1        250.0    250.0      0.0          p_end = timestamps[end_frame]
     8         1        140.0    140.0      0.0          start_date = timestamps[0]
   

In [15]:

def create_matrix_point( result_queue, begin_frame, end_frame, time_delta_size, extent, timestamps, connection_parameters, granularity_enum, srid, ids_str):
    try:
        logs=""
        pid = os.getpid()
        logs += (f"pid : {pid} create_matrix |  CPU affinity : {psutil.Process(pid).cpu_affinity()} \n") 
        p_start = timestamps[begin_frame]
        p_end = timestamps[end_frame]
        start_date = timestamps[0]
        x_min,y_min, x_max, y_max = extent
        logs = ""
        
        # Part 1 : Fetch Tpoints from MobilityDB database
        connection_params = {
            "host": connection_parameters["host"],
            "port": connection_parameters["port"],
            "dbname": connection_parameters["dbname"],
            "user": connection_parameters["user"],
            "password": connection_parameters["password"]
        }
        table_name = connection_parameters["table_name"]
        id_column_name = connection_parameters["id_column_name"]
        tpoint_column_name = connection_parameters["tpoint_column_name"]

        connection = MobilityDB.connect(**connection_params)    
        cursor = connection.cursor()
    
        if granularity_enum.value["name"] == "SECOND": 
            time_value = 1 * granularity_enum.value["steps"]
        elif granularity_enum.value["name"] == "MINUTE":
            time_value = 60 * granularity_enum.value["steps"]

        query = f"""WITH trajectories as (
                SELECT 
                    atStbox(
                        a.{tpoint_column_name}::tgeompoint,
                        stbox(
                            ST_MakeEnvelope(
                                {x_min}, {y_min}, -- xmin, ymin
                                {x_max}, {y_max}, -- xmax, ymax
                                {srid} -- SRID
                            ),
                            tstzspan('[{p_start}, {p_end}]')
                        )
                    ) as trajectory
                FROM public.{table_name} as a 
                WHERE a.{id_column_name} in ({ids_str})),

                resampled as (

                SELECT tsample(traj.trajectory, INTERVAL '{granularity_enum.value["steps"]} {granularity_enum.value["name"]}', TIMESTAMP '{start_date}')  AS resampled_trajectory
                    FROM 
                        trajectories as traj)
            
                SELECT
                        EXTRACT(EPOCH FROM (startTimestamp(rs.resampled_trajectory) - '{start_date}'::timestamp))::integer / {time_value} AS start_index ,
                        EXTRACT(EPOCH FROM (endTimestamp(rs.resampled_trajectory) - '{start_date}'::timestamp))::integer / {time_value} AS end_index,
                        rs.resampled_trajectory
                FROM resampled as rs ;"""

        cursor.execute(query)
        # logs += f"query : {query}\n"
        rows = cursor.fetchall()
        cursor.close()
        connection.close()

        logs += f"Number of rows : {len(rows)}\n"
        now_matrix =time.time()
        empty_point = Point()  # "POINT EMPTY"
        matrix = np.full((len(rows), time_delta_size), empty_point, dtype=object)
        
        
        rows_to_remove = []
        for i in range(len(rows)):
            if rows[i][2] is not None:
                try:
                    traj_resampled = rows[i][2]

                    start_index = rows[i][0] - begin_frame
                    end_index = rows[i][1] - begin_frame
                    values = np.array(traj_resampled.values())
                    matrix[i, start_index:end_index+1] = values
            
                except Exception as e:
                    logs += f"Error at row {i} : {e}\n"
                    break
            else:
                rows_to_remove.append(i)
        
        matrix = np.delete(matrix, rows_to_remove, axis=0) 

        logs += f"Matrix generation time : {time.time() - now_matrix}\n"
        logs += f"Matrix shape : {matrix.shape}\n"
        logs += f"Number of non empty points : {np.count_nonzero(matrix != 'POINT EMPTY')}\n"

        result_queue.put(0)
        result_queue.put(matrix)
        result_queue.put(logs)

    except Exception as e:
        result_queue.put(1)
        result_queue.put(e)
        result_queue.put(logs)
        return False
    

In [16]:
%lprun -f create_matrix_point create_matrix_point(result_queue, begin_frame, end_frame, TIME_DELTA_SIZE, extent_list, timestamps, connection_parameters, GRANULARITY, SRID, objects_id_str)

Timer unit: 1e-09 s

Total time: 3.74407 s
File: /tmp/ipykernel_14199/2728783165.py
Function: create_matrix_point at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def create_matrix_point( result_queue, begin_frame, end_frame, time_delta_size, extent, timestamps, connection_parameters, granularity_enum, srid, ids_str):
     2         1        550.0    550.0      0.0      try:
     3         1        130.0    130.0      0.0          logs=""
     4         1        970.0    970.0      0.0          pid = os.getpid()
     5         1     231288.0 231288.0      0.0          logs += (f"pid : {pid} create_matrix |  CPU affinity : {psutil.Process(pid).cpu_affinity()} \n") 
     6         1        440.0    440.0      0.0          p_start = timestamps[begin_frame]
     7         1        250.0    250.0      0.0          p_end = timestamps[end_frame]
     8         1        170.0    170.0      0.0          start_date = times

# ONF memory profiling:

In [ ]:
from line_profiler import LineProfiler

def on_new_frame(current_matrix, vlayer):
    """
    Handles the logic at each frame change.
    """

    # frame_number = self.move_layer_controller.get_current_frame_number()
    frame_number = 1

    # if self.previous_frame - frame_number <= 0:
    #     self.direction = 1 # Forward
    #     if frame_number == 780: # Reached the end of the animation, pause
    #         self.move_layer_controller.pause()

    # else:
    #     self.direction = 0
    #     if frame_number <= 0: # Reached the beginning of the animation, pause
    #         self.move_layer_controller.pause()
        


    time_delta_key = 0
  
    frame_index = frame_number- time_delta_key


    current_time_stamp_column = current_matrix[:, frame_index]

    new_geometries = {}  
    # new_geometries = {QgsGeometry().fromWkt(point) for point in current_time_stamp_column}  # Dictionary {feature_id: QgsGeometry}
    for i in range(current_time_stamp_column.shape[0]): #TODO : compare vs Nditer
        new_geometries[i] = QgsGeometry().fromWkt(current_time_stamp_column[i])


    vlayer.startEditing()
    # self.move_layer_controller.vlayer.dataProvider().changeAttributeValues(attribute_changes) # Updating attribute values for all features
    vlayer.dataProvider().changeGeometryValues(new_geometries) # Updating geometries for all features
    vlayer.commitChanges()
    iface.vectorLayerTools().stopEditing(vlayer)



In [1]:
from shapely.geometry import Point
import numpy as np

point_wkt = Point(1,4).wkt

num_fields = 3000
matrix = np.full((num_fields, 60), point_wkt, dtype=object)
matrix


vlayer = QgsVectorLayer("Point", "name", "memory")
pr = vlayer.dataProvider()
pr.addAttributes([QgsField("start_time", QVariant.DateTime), QgsField("end_time", QVariant.DateTime)])
vlayer.updateFields()
tp = vlayer.temporalProperties()
tp.setIsActive(True)
tp.setMode(qgis.core.QgsVectorLayerTemporalProperties.ModeFeatureDateTimeStartAndEndFromFields)
tp.setStartField("start_time")
tp.setEndField("end_time")
QgsProject.instance().addMapLayer(vlayer)


features_list =[]
vfields = vlayer.fields()
start_datetime_obj = QDateTime(datetime(2023,6,1,0,0,0))
end_datetime_obj = QDateTime(datetime(2023,6,1,23,0,0))


for i in range(num_fields):
    feat = QgsFeature(vfields)
    feat.setAttributes([start_datetime_obj, end_datetime_obj])
    features_list.append(feat)

pr.addFeatures(features_list)



lp = LineProfiler()
lp.add_function(on_new_frame)

# Run the function with the profiler
lp_wrapper = lp(on_new_frame)
lp_wrapper(matrix, vlayer)

# Print the profiling results
lp.print_stats()
with open("profile_results.txt", "w") as stream:
    lp.print_stats(stream)

array([['POINT (1 4)', 'POINT (1 4)', 'POINT (1 4)', ..., 'POINT (1 4)',
        'POINT (1 4)', 'POINT (1 4)'],
       ['POINT (1 4)', 'POINT (1 4)', 'POINT (1 4)', ..., 'POINT (1 4)',
        'POINT (1 4)', 'POINT (1 4)'],
       ['POINT (1 4)', 'POINT (1 4)', 'POINT (1 4)', ..., 'POINT (1 4)',
        'POINT (1 4)', 'POINT (1 4)'],
       ...,
       ['POINT (1 4)', 'POINT (1 4)', 'POINT (1 4)', ..., 'POINT (1 4)',
        'POINT (1 4)', 'POINT (1 4)'],
       ['POINT (1 4)', 'POINT (1 4)', 'POINT (1 4)', ..., 'POINT (1 4)',
        'POINT (1 4)', 'POINT (1 4)'],
       ['POINT (1 4)', 'POINT (1 4)', 'POINT (1 4)', ..., 'POINT (1 4)',
        'POINT (1 4)', 'POINT (1 4)']], dtype=object)

Timer unit: 1e-09 s

Total time: 0.00621436 s

Could not find file <string>
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           
     4                                           
     5                                           
     6                                           
     7                                           
     8                                           
     9         1       1050.0   1050.0      0.0  
    10                                           
    11                                           
    12                                           
    13                                           
    14                                           
    15                                           
    16                                           
    17                                           
    18                                           
    19                                           
    20                                           
    21                                           
    22                                           
    23         1        120.0    120.0      0.0  
    24                                           
    25         1        440.0    440.0      0.0  
    26                                           
    27                                           
    28         1       3740.0   3740.0      0.1  
    29                                           
    30         1        280.0    280.0      0.0  
    31                                           
    32      3001     261278.0     87.1      4.2  
    33      3000    3887763.0   1295.9     62.6  
    34                                           
    35                                           
    36         1     734982.0 734982.0     11.8  
    37                                           
    38         1     444645.0 444645.0      7.2  
    39         1     487715.0 487715.0      7.8  
    40         1     392346.0 392346.0      6.3  

